# Fine-Tuning and Evaluating LLMs with SageMaker Pipelines and MLflow

In [13]:
!pip install sagemaker==2.225.0  datasets==2.18.0 transformers==4.40.0 mlflow==2.13.2 sagemaker-mlflow==0.1.0 --quiet

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.function_step import step
from steps.finetune_llama8b_hf import finetune_llama8b
from steps.preprocess_llama3 import preprocess
from steps.eval_mlflow import evaluation
from steps.utils import create_training_job_name
import os

os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

## 1. SageMaker Session & IAM Role

In [22]:
import boto3

try:
    role = sagemaker.get_execution_role()
    print(role)
except ValueError:
    iam = boto3.client("iam")
    # Hard coded ARN since, I'm running this notebook locally to reduce AWS costs
    role = "arn:aws:iam::891612587330:role/service-role/AmazonSageMaker-ExecutionRole-20241230T123665"

sess = sagemaker.Session()

## 2. Training Configurations 

In [23]:
train_config = {
    "experiment_name": "all_target_modules_1K",
    "model_id": "meta-llama/Meta-Llama-3-8B",
    "model_version": "3.0.2",
    "model_name": "llama-3-8b",
    "endpoint_name": "llama-3-8b",
    "finetune_instance_type": "ml.g5.12xlarge",
    "finetune_num_instances": 1,
    "instance_type": "ml.g5.12xlarge",
    "num_instances": 1,
    "epoch": 1,
    "per_device_train_batch_size": 4,
}

### LoRA Parameters

In [25]:
lora_params = {"lora_r": 8, "lora_alpha": 16, "lora_dropout": 0.05}

### MLFlow Setup

In [26]:
mlflow_arn = "arn:aws:sagemaker:us-east-2:891612587330:mlflow-tracking-server/llm-finetuning-experiment"  # fill MLflow tracking server ARN
experiment_name = "llm-finetuning-experiment"

### Dataset Configurations 

In [27]:
dataset_name = "HuggingFaceH4/no_robots"

### Setting up Pipeline Steps

In [28]:
from sagemaker.workflow.parameters import ParameterString
import json

In [29]:
lora_config = ParameterString(name="lora_config", default_value=json.dumps(lora_params))

#### Preprocessing step

In [31]:
pipeline_name = "training-evaulation-pipeline-mlflow"

default_bucket = sagemaker.Session().default_bucket()
main_data_path = f"s3://{default_bucket}"
evaluation_data_path = (
    main_data_path
    + "/datasets/hf_no_robots/evaluation/automatic_small/dataset_evaluation_small.jsonl"
)
output_data_path = main_data_path + "/datasets/hf_no_robots/output_" + pipeline_name

# You can add your own evaluation dataset code into this step
preprocess_step_ret = step(preprocess, name="preprocess")(
    default_bucket,
    dataset_name,
    train_sample=100,
    eval_sample=100,
    mlflow_arn=mlflow_arn,
    experiment_name=experiment_name,
    run_name=ExecutionVariables.PIPELINE_EXECUTION_ID,
)

print("The pipeline name is " + pipeline_name)
# Mark the name of this bucket for reviewing the artifacts generated by this pipeline at the end of the execution
print("Output S3 bucket: " + output_data_path)

The pipeline name is training-evaulation-pipeline-mlflow
Output S3 bucket: s3://sagemaker-us-east-1-711387114394/datasets/hf_no_robots/output_training-evaulation-pipeline-mlflow


#### Fine-tuning step

In [32]:
finetune_ret = step(finetune_llama8b, name="finetune_llama8b_instruction")(
    preprocess_step_ret,
    train_config,
    lora_config,
    role,
    mlflow_arn,
    experiment_name,
    ExecutionVariables.PIPELINE_EXECUTION_ID,
)

NameError: name 'finetune_llama7b' is not defined